## Preprocessing

In [53]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [54]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
cleaned_df= application_df.drop(columns= ['EIN', 'NAME'])

In [55]:
# display first 3 rows
cleaned_df.head(3)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0


In [56]:
# Determine the number of unique values in each column.
columns_unique_values = cleaned_df.nunique(axis=0)
print(columns_unique_values)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [57]:
# Look at APPLICATION_TYPE value counts for binning
application_type_value_count = cleaned_df.APPLICATION_TYPE.value_counts()
application_type_value_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [58]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff = application_type_value_count['T10']
application_types_to_replace = application_type_value_count[application_type_value_count < cutoff].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [59]:
# Look at CLASSIFICATION value counts for binning
class_count = application_df['CLASSIFICATION'].value_counts()
class_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C3700        1
C4200        1
C2190        1
C4120        1
C2380        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [60]:
# You may find it helpful to look at CLASSIFICATION value counts >1
count_greater_than_one = class_count[class_count > 1]
count_greater_than_one

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C4100        6
C1720        6
C1257        5
C1600        5
C2710        3
C0           3
C1260        3
C1267        2
C3200        2
C1246        2
C1234        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [61]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = class_count['C2100']
classifications_to_replace = class_count[class_count < cutoff].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()


C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [62]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies_df = pd.get_dummies(application_df)

In [63]:
# Split our preprocessed data into our features and target arrays
y = dummies_df['IS_SUCCESSFUL']
X = dummies_df.drop(['IS_SUCCESSFUL'], axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78) 

In [64]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# OPTIMIZATION ATTEMPT NO. 1

## Compile, Train and Evaluate the Model

In [65]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_ft = len(X_train_scaled[0])

# In the original program we got a 76% accuracy, I'll try to attempt to get a highter accuracy %
hidden_1 = 6
hidden_2 = 2
hidden_3 = 2

In [66]:
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_1, activation="relu", input_dim=input_ft))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_2, activation="relu", input_dim=input_ft))

# Adding a third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_3, activation="relu", input_dim=input_ft))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# # Check the structure of the model
nn.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_26 (Dense)            (None, 6)                 117678    
                                                                 
 dense_27 (Dense)            (None, 2)                 14        
                                                                 
 dense_28 (Dense)            (None, 2)                 6         
                                                                 
 dense_29 (Dense)            (None, 1)                 3         
                                                                 
Total params: 117,701
Trainable params: 117,701
Non-trainable params: 0
_________________________________________________________________


In [67]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [68]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 6s 5ms/step - loss: 0.6407 - accuracy: 0.6542
Epoch 2/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5382 - accuracy: 0.7457
Epoch 3/100
804/804 [==============================] - 4s 5ms/step - loss: 0.3576 - accuracy: 0.8497
Epoch 4/100
804/804 [==============================] - 5s 6ms/step - loss: 0.2634 - accuracy: 0.9036
Epoch 5/100
804/804 [==============================] - 5s 6ms/step - loss: 0.2264 - accuracy: 0.9191
Epoch 6/100
804/804 [==============================] - 5s 6ms/step - loss: 0.2168 - accuracy: 0.9227
Epoch 7/100
804/804 [==============================] - 5s 6ms/step - loss: 0.2146 - accuracy: 0.9229
Epoch 8/100
804/804 [==============================] - 4s 5ms/step - loss: 0.2124 - accuracy: 0.9226
Epoch 9/100
804/804 [==============================] - 4s 5ms/step - loss: 0.2095 - accuracy: 0.9226
Epoch 10/100
804/804 [==============================] - 4s 5ms/step - loss: 0.2069 - accura

In [69]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6861 - accuracy: 0.6134 - 872ms/epoch - 3ms/step
Loss: 0.6861225366592407, Accuracy: 0.6134110689163208


Accuracy of 61% - needs more improvement

# OPTIMIZATION ATTEMPT NO. 2

In [70]:
nn2 = tf.keras.models.Sequential()

# TWO HIDDEN LAYERS
# First hidden layer
nn2.add(tf.keras.layers.Dense(units=8, activation="relu", input_dim=input_ft))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=2, activation="relu", input_dim=input_ft))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 5s 4ms/step - loss: 0.6584 - accuracy: 0.6952
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.4892 - accuracy: 0.7081
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.3630 - accuracy: 0.8024
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.2926 - accuracy: 0.8689
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 0.2117 - accuracy: 0.9350
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.1544 - accuracy: 0.9546
Epoch 7/100
804/804 [==============================] - 4s 4ms/step - loss: 0.1332 - accuracy: 0.9584
Epoch 8/100
804/804 [==============================] - 3s 4ms/step - loss: 0.1296 - accuracy: 0.9546
Epoch 9/100
804/804 [==============================] - 3s 4ms/step - loss: 0.1133 - accuracy: 0.9612
Epoch 10/100
804/804 [==============================] - 4s 4ms/step - loss: 0.1081 - accura

In [71]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5142 - accuracy: 0.7860 - 1s/epoch - 5ms/step
Loss: 0.5141658782958984, Accuracy: 0.7860058546066284


Accuracy of 57% - lost 4% accuracy from attempt No. 2

# OPTIMIZATION ATTEMPT NO. 3

In [72]:
nn3 = tf.keras.models.Sequential()

# TWO HIDDEN LAYERS
# First hidden layer
nn3.add(tf.keras.layers.Dense(units=7, activation="relu", input_dim=input_ft))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=3, activation="relu", input_dim=input_ft))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn3.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 31s 6ms/step - loss: 1.3409 - accuracy: 0.5957
Epoch 2/100
804/804 [==============================] - 4s 5ms/step - loss: 0.4764 - accuracy: 0.8034
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.3256 - accuracy: 0.9169
Epoch 4/100
804/804 [==============================] - 4s 5ms/step - loss: 0.2293 - accuracy: 0.9413
Epoch 5/100
804/804 [==============================] - 4s 5ms/step - loss: 0.1920 - accuracy: 0.9479
Epoch 6/100
804/804 [==============================] - 4s 5ms/step - loss: 0.1666 - accuracy: 0.9514
Epoch 7/100
804/804 [==============================] - 4s 5ms/step - loss: 0.1508 - accuracy: 0.9532
Epoch 8/100
804/804 [==============================] - 4s 5ms/step - loss: 0.1438 - accuracy: 0.9543
Epoch 9/100
804/804 [==============================] - 4s 5ms/step - loss: 0.1398 - accuracy: 0.9546
Epoch 10/100
804/804 [==============================] - 4s 4ms/step - loss: 0.1378 - accur

In [73]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5671 - accuracy: 0.7053 - 852ms/epoch - 3ms/step
Loss: 0.5670809745788574, Accuracy: 0.7053061127662659


In [74]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization(h5).h5')